# 3.3 Nonlinear problems

In this unit we turn our attention to nonlinear PDE problems and the tools that `NGSolve` provides to simplify it.

## A simple scalar PDE
Let us start with a simple PDE with a nonlinearity: 

$$
- \Delta u + \frac13 u^3 = 10 \text{ in } \Omega
$$

on the unit square $\Omega = (0,1)^2$. 

We note that this PDE can also be formulated as a nonlinear minimization problem (cf. [3.4](../unit-3.4-nonlmin/nonlmin.ipynb)).

In [ ]:
# define geometry and generate mesh
from ngsolve import *
from ngsolve.webgui import *
from netgen.occ import *
shape = Rectangle(1,1).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
geom = OCCGeometry(shape, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=0.3))

In NGSolve we can solve the PDE conveniently using the *linearization* feature of `SymbolicBFI`.

The `BilinearForm` (**which is not bilinear!**) needed in the weak formulation is
$$
  A(u,v) = \int_{\Omega} \nabla u \nabla v + 1/3 u^3 v - 10 v ~ dx \quad ( = 0 ~ \forall~v \in H^1_0)
$$

In [ ]:
V = H1(mesh, order=3, dirichlet=[1,2,3,4])
u,v = V.TnT()
a = BilinearForm(V)
a += (grad(u) * grad(v) + 1/3*u**3*v- 10 * v)*dx

### Newton's method

In the preparation of the formulation of Newton's method we identify the semilinear form $A(\cdot,\cdot)$ with the corresponding operator $A: V \to V'$ and write (assuming sufficient regularity)

 * $A(u)$ for the linear form $[A(u)](v) = A(u,v)$ and 
 * the derivative of $A$ at an evaluation point $u$ is the bilinear form $\delta A(u)$ with
 $$[\delta A(u)] (w,v) = \lim_{h \to 0} \frac{ A(u+h\cdot w,v) - A(u,v)}{h}$$


You could compute these linear and bilinear forms manually. However, `NGSolve` provides functions for both operations: 

* The linear form $A(u)$ for a given (vector) u is obtained from `a.Apply(vec,res)` (resulting in a vector `res` representing the linear form)
* The bilinear form (represented by the corresponding matrix) is stored in `a.mat` after a call of `a.AssembleLinearization(vec)`

Under the hood it uses its functionality to derive `CoefficientFunction`s symbolically. 

For example, `NGSolve` derives the bilinear form integrand $ \frac13 u^3 v $ w.r.t. $u$ at $\tilde{u}$ in direction $w$ resulting in the integrand $\tilde{u}^2 w v$.

This allows to form the corresponding bilinear form integrals automatically for you.

To obtain a Newton algorithm we hence only need to translate the following pseudo-code formulation of Newton's method to `NGSolve`. The pseudo code is:

`NewtonSolve`(Pseudo code):

* Given an initial guess $u^0$
* loop over $i=0,..$ until convergence:
  * Compute linearization: $A u^i + \delta A(u^i) \Delta u^{i} = 0$:
    * $f^i = A u^i$ 
    * $B^i = \delta A(u^i)$ 
    * Solve $B^i \Delta u^i = -f^i$
  * Update $u^{i+1} = u^i + \Delta u^{i}$
  * Evaluate stopping criteria

As a stopping criteria we take $\langle A u^i,\Delta u^i \rangle = \langle A u^i, A u^i \rangle_{(B^i)^{-1}}< \varepsilon$.

Now, here comes the same thing in `NGSolve` syntax:

In [ ]:
def SimpleNewtonSolve(gfu,a,tol=1e-13,maxits=10, callback=lambda gfu: None):
    res = gfu.vec.CreateVector()
    du = gfu.vec.CreateVector()
    fes = gfu.space
    callback(gfu)
    for it in range(maxits):
        print ("Iteration {:3}  ".format(it),end="")
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec)
        du.data = a.mat.Inverse(fes.FreeDofs()) * res
        gfu.vec.data -= du
        callback(gfu)
        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break

Let's apply this to the previous PDE problem:

In [ ]:
gfu = GridFunction(V)
gfu.Set((x*(1-x))**4*(y*(1-y))**4) # initial guess
gfu_it = GridFunction(gfu.space,multidim=0)
cb = lambda gfu : gfu_it.AddMultiDimComponent(gfu.vec) # store current state
SimpleNewtonSolve(gfu, a, callback = cb)

In [ ]:
Draw(gfu,mesh,"u", deformation = True)

In [ ]:
Draw(gfu_it,mesh,"u", deformation = True)

Use the `multidim`-Slider to inspect the results after the iterations.

There are also some solvers shipped with NGSolve now. The `ngsolve.solvers.Newton` method allows you to incorporate Dirichlet boundary conditions, use static condensation, and is overall more refined (with damping options and line searches). For this tutorial we will mostly stay with the simple hand-crafted `SimpleNewtonSolve`. 
Here is the alternative demonstrated once, nevertheless:


In [ ]:
from ngsolve.solvers import *
help(Newton)

We call this Newton method as well here. 

In [ ]:
gfu.Set((x*(1-x))**4*(y*(1-y))**4) # initial guess
Newton(a,gfu,freedofs=gfu.space.FreeDofs(),maxit=100,maxerr=1e-11,inverse="umfpack",dampfactor=1,printing=True)

## A trivial problem:
As a second simple problem, let us consider a trivial scalar problem:
$$
  5 u^2 = 1, \qquad u \in \mathbb{R}.
$$

We chose this problem and put this in the (somewhat artificially) previous PDE framework and solve with Newton's method in a setting that you could easily follow with pen and paper:

In [ ]:
V = NumberSpace(mesh)
u,v = V.TnT()
a = BilinearForm(V)
a += ( 5*u*u*v - 1 * v)*dx
gfu = GridFunction(V)
gfu.vec[:] = 1
SimpleNewtonSolve(gfu,a, callback = lambda gfu : print(f"u^k = {gfu.vec[0]}, u^k**2 = {gfu.vec[0]**2}"))
print(f"\nscalar solution, {gfu.vec[0]}, exact: {sqrt(0.2)}, error: {abs(sqrt(0.2)-gfu.vec[0])}")

## Another example: Stationary Navier-Stokes:

Next, we consider incompressible Navier-Stokes equations again. This time however stationary.

Find $\mathbf{u} \in \mathbf{V}$, $p \in Q$, $\lambda \in \mathbb{R}$ so that

\begin{align*}
\int_{\Omega} \nu \nabla \mathbf{u} : \nabla \mathbf{v} + (\mathbf{u} \cdot \nabla) \mathbf{u} \cdot \mathbf{v}& - \int_{\Omega} \operatorname{div}(\mathbf{v}) p & &= \int \mathbf{f}  \cdot \mathbf{v}  && \forall \mathbf{v} \in \mathbf{V}, \\ 
- \int_{\Omega} \operatorname{div}(\mathbf{u}) q & & 
+ \int_{\Omega} \lambda q
&= 0 && \forall q \in Q, \\
& \int_{\Omega} \mu p & &= 0 && \forall \mu \in \mathbb{R}.
\end{align*}



The domain $\Omega$ is still $(0,1)^2$ and we prescribe homogenuous Dirichlet bnd. conditions for the velocity, except for the top boundary where we prescribe a tangential velocity. This setup is known as "driven cavity".

Note that we use a scalar constraint to fix the pressure level that is otherwise not controlled in the presence of pure Dirichlet conditions.

We use a higher order Taylor-Hood discretization again:

In [ ]:
mesh = Mesh (geom.GenerateMesh(maxh=0.05)); nu = Parameter(1)
V = VectorH1(mesh,order=3,dirichlet="bottom|right|top|left")
Q = H1(mesh,order=2); 
N = NumberSpace(mesh); 
X = V*Q*N
(u,p,lam), (v,q,mu) = X.TnT()
a = BilinearForm(X)
a += (nu*InnerProduct(grad(u),grad(v))+InnerProduct(grad(u)*u,v)
      -div(u)*q-div(v)*p-lam*q-mu*p)*dx

The boundary condition:

In [ ]:
gfu = GridFunction(X)
gfu.components[0].Set(CF((4*x*(1-x),0)),
                      definedon=mesh.Boundaries("top"))

Now, let's apply the Newton:

In [ ]:
def SolveAndVisualize(multidim=True):
    gfu.components[0].Set(CF((4*x*(1-x),0)),
                      definedon=mesh.Boundaries("top"))
    if multidim:
        gfu_it = GridFunction(gfu.space,multidim=0)
        cb = lambda gfu : gfu_it.AddMultiDimComponent(gfu.vec) # store current state
        SimpleNewtonSolve(gfu, a, callback = cb)
    else:
        SimpleNewtonSolve(gfu, a)
    Draw(gfu.components[0],mesh, vectors = {"grid_size" : 25})
    print("above you see the solution after the Newton solve.")
    if multidim:
        Draw(gfu_it.components[0], mesh, vectors = {"grid_size" : 25})
        print("above you can inspect the results after each iteration of the Newton solve (use multidim-slider).")

In [ ]:
SolveAndVisualize()

The problem becomes more interesting if we decrease the viscosity $\nu$ (`nu`), i.e. if we increase the Reynolds number *Re*. Let's consider the previous setup for decreasing values of `nu`

In [ ]:
nu.Set(0.01)
SolveAndVisualize(multidim=False)

In [ ]:
nu.Set(0.01)
SolveAndVisualize()

In [ ]:
nu.Set(0.001)
SolveAndVisualize()

Now, viscosity is so small that Newton does not converge anymore.
In this case we can fix it using a small damping parameter with the `ngsolve.solvers.Newton`-solver:

In [ ]:
nu.Set(0.001)
gfu.components[0].Set(CF((4*x*(1-x),0)),definedon=mesh.Boundaries("top"))
Newton(a,gfu,maxit=20,dampfactor=0.1)
Draw(gfu.components[0],mesh, vectors = {"grid_size" : 25})

**Tasks** 

After these explanations, here are a few suggestions for simple play-around-tasks:

* Take the first PDE example and set up the linearization linear and bilinear forms by hand and implement a Newton solver without exploiting the convenience functions `NGSolve` provides to you.
* Combine [unit 3.1](../unit-3.1-parabolic/parabolic.ipynb) and this unit and write an implicit time integration solver for the unsteady Navier-Stokes equations (start with an implicit Euler)